In [ ]:
#! pip install fairlearn
#! pip install lightgbm
#! pip install optuna

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, make_scorer, accuracy_score, recall_score
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from fairlearn.datasets import fetch_adult
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

import numpy as np
import optuna
import tqdm as notebook_tqdm
from metrics import (
    equality_opportunity_difference,
    predictive_equality_difference,
    metrics
)

/home/azucena/miniconda3/envs/fairenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1
A = X_raw["sex"]

numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LGBMClassifier(n_jobs=-1),
        ),
    ]
)

/home/azucena/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
results = []

In [ ]:

directions = ['minimize', 'maximize']
metric_scorer_decorated =  metrics(recall_score, predictive_equality_difference, sensitive_col = 'sex')
for sim in [0,1,2,3]:
    print(sim)
    def objective(trial):

        (X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
        X_raw, y, A, test_size=0.8, random_state=sim, stratify=y
        )

        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
        A_train = A_train.reset_index(drop=True)
        A_test = A_test.reset_index(drop=True)


        params = {
            'n_estimators': trial.suggest_int("n_estimators", 20, 10000),
            'num_leaves': trial.suggest_int("num_leaves", 10, 1000),
            'max_depth': trial.suggest_int("max_depth", 2, 20),
            'min_child_samples': trial.suggest_int("min_child_samples", 5, 300),
            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-2),
            'boosting_type': trial.suggest_categorical("boosting_type", ['goss', 'gbdt'])
            }
        pipeline['classifier'].set_params(**params)

        scores = cross_validate(
                pipeline, 
                X_train,
                y_train, 
                cv=5,
                scoring = metric_scorer_decorated,
                return_train_score=True)

        fair_metric = scores['test_fairness'].mean()
        model_metric = scores['test_model'].mean()

        return fair_metric, model_metric
    
    
    
    study = optuna.create_study(
        directions = directions, 
        pruner = optuna.pruners.SuccessiveHalvingPruner(), 
        sampler = optuna.samplers.TPESampler() 
        )
    study.optimize(objective, n_trials=100)
    print("Number of finished trials: ", len(study.trials))
    results.append(study)

[I 2023-05-12 14:56:31,969] A new study created in memory with name: no-name-14f82152-8e34-4dde-b4c7-9cb70b032495


1


[I 2023-05-12 14:56:42,430] Trial 0 finished with values: [0.09479653381492684, 0.6106187887772471] and parameters: {'n_estimators': 2724, 'num_leaves': 305, 'max_depth': 7, 'min_child_samples': 129, 'learning_rate': 0.009684131645262545, 'boosting_type': 'goss'}. 
[I 2023-05-12 14:57:02,316] Trial 1 finished with values: [0.09518159564146676, 0.5994802247478908] and parameters: {'n_estimators': 5255, 'num_leaves': 275, 'max_depth': 8, 'min_child_samples': 201, 'learning_rate': 0.002453173913138281, 'boosting_type': 'goss'}. 
[I 2023-05-12 14:57:37,242] Trial 2 finished with values: [0.09560429655359028, 0.6067689745419939] and parameters: {'n_estimators': 8061, 'num_leaves': 136, 'max_depth': 12, 'min_child_samples': 170, 'learning_rate': 0.00837787045415179, 'boosting_type': 'goss'}. 
[I 2023-05-12 14:59:10,245] Trial 3 finished with values: [0.10002034774009505, 0.6401306758908472] and parameters: {'n_estimators': 5954, 'num_leaves': 713, 'max_depth': 20, 'min_child_samples': 95, 'l

Object successfully saved to "recall-fpr-lgbm-motpe-succesivehalving-100trials-1sim.pkl"


In [7]:
optuna.visualization.plot_pareto_front(study, target_names=["fair_metric", "f1_score"], include_dominated_trials = True)